# Looking at https://github.com/ytsvetko/metaphor

In [1]:
##all the imports...
#%pylab inline
import re
import math
import string
import nltk
import pprint
import matplotlib
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
from collections import Counter
from __future__ import division
from nltk.collocations import *
import string, random
from nltk.corpus import brown
from nltk.collocations import *
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk import bigrams
from nltk import collocations
from nltk import trigrams
from nltk.corpus import stopwords
from nltk.corpus import brown

## Start by loading the .csv files and setting the columns.
### For these .csv files, downloaded from https://github.com/ytsvetko/metaphor inputs and saved as .csv with the first row added as "sample"

In [2]:
df_anmet = pd.read_csv("an_mets.csv", low_memory=False)
df_anmet.head()

,sample
0,angry welt
1,bald assertion
2,bare outline
3,black humor
4,blind alley


In [3]:
df_anmet = pd.read_csv("an_mets.csv", low_memory=False)
df_anmet['metaphor'] = 1
df_anmet['an'] = 1
df_anmet['svo'] = 0
df_anmet['metanet'] = 0
df_anmet.head()
len(df_anmet)

100

In [4]:
df_annonmet = pd.read_csv("an_nonmets.csv", low_memory=False)
df_annonmet['metaphor'] = 0
df_annonmet['an'] = 1
df_annonmet['svo'] = 0
df_annonmet['metanet'] = 0
df_annonmet.head()
len(df_annonmet)

100

In [5]:
df_svomet = pd.read_csv("svo_mets.csv", low_memory=False)
df_svomet['metaphor'] = 1
df_svomet['an'] = 0
df_svomet['svo'] = 1
df_svomet['metanet'] = 0
df_svomet.head()
len(df_svomet)

111

In [6]:
df_svononmet = pd.read_csv("svo_nonmets.csv", low_memory=False)
df_svononmet['metaphor'] = 0
df_svononmet['an'] = 0
df_svononmet['svo'] = 1
df_svononmet['metanet'] = 0
df_svononmet.head()
len(df_svononmet)

111

In [7]:
df_metanet = pd.read_csv("metanet.csv", low_memory=False)
df_metanet['metaphor'] = 1
df_metanet['an'] = 0
df_metanet['svo'] = 0
df_metanet['metanet'] = 1
df_metanet.head()
# len(df_metanet)

,sample,metaphor,an,svo,metanet
0,ability to evaluate government is ability to see,1,0,0,1
1,ability to evaluate is ability to see,1,0,0,1
2,ability to know is ability to see,1,0,0,1
3,abusive political leaders are physical bullies,1,0,0,1
4,accepting is swallowing,1,0,0,1


In [8]:
#combine into one df
frames = [df_anmet, df_annonmet, df_svomet, df_svononmet, df_metanet]
df_combo = pd.concat(frames)
df_combo.reset_index(drop=True, inplace=True)
df_combo.shape
# df_combo.replace(to_replace='none', value="")
# #         tokenizer = RegexpTokenizer(r'\w+')
# #         word_list = tokenizer.tokenize(line)
# #         filtered_words = [word for word in word_list if word not in stopwords.words('english')]
# df_combo.replace(to_replace='is', value="")
stop = ['*none', '.', 'is']
df_combo['sample'].apply(lambda x: [item for item in x if item not in stop])
df_combo[200:500]

,sample,metaphor,an,svo,metanet
200,conversation turn subject,1,0,1,0
201,resumption bring relief,1,0,1,0
202,economy move direction,1,0,1,0
203,service meet expectation,1,0,1,0
204,material live dream,1,0,1,0
205,unemployment stand *none,1,0,1,0
206,action talk *none,1,0,1,0
207,statement sit *none,1,0,1,0
208,income fall *none,1,0,1,0
209,silence speak volume,1,0,1,0


## Break down into training, dev, and test sets

In [9]:
random_index = np.random.permutation(df_combo.index)
df_combo.ix[random_index, ['sample', 'metaphor', 'an', 'svo', 'metanet']]
df_shuffled = df_combo.ix[random_index, ['sample', 'metaphor', 'an', 'svo', 'metanet']]
df_shuffled.reset_index(drop=True, inplace=True)
len(df_shuffled)

1606

In [10]:
rows, columns = df_shuffled.shape
print("Rows:", rows)
print("Columns:", columns)
#train_size = round(rows*.6)
train_size = round(rows*.9)
#dev_size   = round(rows*.2)
dev_size   = round(rows*.1)
df_train = df_shuffled.loc[:train_size]
df_train.shape
df_dev = df_shuffled.loc[train_size:dev_size+train_size].reset_index(drop=True)
df_dev.shape
df_test = df_shuffled.loc[dev_size+train_size:].reset_index(drop=True)
df_test.shape

Rows: 1606
Columns: 5


(0, 5)

## CountVectorizer
### Use count vecotrizer with df = 3 for unigram and bigrams to predict dev set

In [11]:
vocab = list(set(brown.words()))

In [12]:
# vec = CountVectorizer(ngram_range=(1, 3), token_pattern=r'\b\w+\b', analyzer=u'word', min_df=1, vocabulary=vocab)
# df_train = df_train.fillna("")
# df_dev = df_dev.fillna("")
# df_test = df_test.fillna("")


vec = TfidfVectorizer(ngram_range=(2, 3), token_pattern=r'\b\w+\b', analyzer=u'char', min_df=1)
df_train = df_train.fillna("")
df_dev = df_dev.fillna("")
df_test = df_test.fillna("")

In [13]:
arr_train_feature_sparse = vec.fit_transform(df_train['sample'])
arr_train_feature_sparse
arr_train_feature = arr_train_feature_sparse.toarray()
feature_labels = vec.get_feature_names()

In [14]:
arr_dev_feature_sparse = vec.transform(df_dev["sample"])
arr_dev_feature = arr_dev_feature_sparse.toarray()

In [15]:
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor']) #defining features (from reviews) and passing in Category label
logreg_predictions = logreg_model.predict(arr_dev_feature)
accuracy_score(df_dev['metaphor'], logreg_predictions)

0.90683229813664601

In [16]:
feature_sum = arr_train_feature.sum(axis=0)   #sum the counts of each feature

df_feature_sum = pd.DataFrame({'counts': feature_sum})
df_feature_sum.index = vec.get_feature_names()
df_feature_sum.sort('counts', ascending=False)[0:10]

/Users/gabe/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,counts
s,56.144560
in,54.688433
i,51.950445
is,50.792850
ti,50.457058
on,50.097886
a,49.599649
is,44.981488
is,44.686130
ng,41.995896


In [17]:
## make a file of sample sentences to practice on
## loop through sentence and .split()
## use nltk bigrams
##
## put into pandas as df_test

In [18]:
word_list = 'Jamal was a pig at dinner'.split()
filtered_words = [word for word in word_list if word not in stopwords.words('english')]
filtered_words
testmeta = list(trigrams(filtered_words))
testmeta
testmeta1 = [' '.join(x) for x in testmeta]
testmeta1
df_test = pd.DataFrame(testmeta1)
df_test
df_test.columns = ['sample']
print(filtered_words)

['Jamal', 'pig', 'dinner']


In [19]:
arr_test_feature_sparse = vec.transform(df_test['sample'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
logreg_predictions

array([1])

In [20]:
df_test = pd.DataFrame(testmeta1)
df_test

,0
0,Jamal pig dinner


In [21]:
with open("test_meta_sent.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
#         word_list = line.split()
        tokenizer = RegexpTokenizer(r'\w+')
        word_list = tokenizer.tokenize(line)
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
#         testmeta1.append(' '.join(filtered_words))
        testmeta1.append(' '.join(x) for x in bigrams(filtered_words))
#         testmeta1.append(' '.join(x) for x in filtered_words)
# print(testmeta1)

df_test =pd.DataFrame(testmeta1)
df_test
df_test = df_test.fillna("")
df_test
# df_test.columns = ['sample']
# df_test = pd.DataFrame(testmeta1)
# df_test
df_test.columns = ['sample0', 'sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6']
# df_test = df_test.fillna("")
# df_test

In [22]:
arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor0'] = logreg_predictions
if df_test['sample0'] == '':
    df_test['metaphor0'] = 0
else:
    df_test['metaphor0'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample1'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor1'] = logreg_predictions
if df_test['sample1'] == '':
    df_test['metaphor1'] = 0
else:
    df_test['metaphor1'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample2'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor2'] = logreg_predictions
if df_test['sample2'] == '':
    df_test['metaphor2'] = 0
else:
    df_test['metaphor2'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample3'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor3'] = logreg_predictions
if df_test['sample3'] == '':
    df_test['metaphor3'] = 0
else:
    df_test['metaphor3'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample4'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor4'] = logreg_predictions
if df_test['sample4'] == '':
    df_test['metaphor4'] = 0
else:
    df_test['metaphor4'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample5'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor5'] = logreg_predictions
if df_test['sample5'] == '':
    df_test['metaphor5'] = 0
else:
    df_test['metaphor5'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample6'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor6'] = logreg_predictions
if df_test['sample6'] == '':
    df_test['metaphor6'] = 0
else:
    df_test['metaphor6'] = logreg_predictions

df_test['sum_metaphor'] = df_test['metaphor0'] + df_test['metaphor1'] + df_test['metaphor2'] + df_test['metaphor3'] +df_test['metaphor4'] +df_test['metaphor5'] + df_test['metaphor6'] 

# def count_metaphors():
#     if sum(df_test['sum_metaphor']) >= 1:
#         df_test['sum_metaphor'] = 1
#     else:
#         df_test['sum_metaphor'] = 0
# count_metaphors()

df_test

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
print(df_test['sample3'] == '')

In [ ]:
df_test['sample3'].str.len()

In [ ]:
with open("an_nonmets.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
        testmeta1.append(' '.join(x) for x in bigrams(filtered_words))
df_test = pd.DataFrame(testmeta1)
df_test
df_test.columns = ['sample0']
df_test = df_test.fillna("")
df_test

arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

sum(df_test['metaphor0'])
#len(df_test['metaphor0'])

In [ ]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
        testmeta1.append(' '.join(x) for x in trigrams(filtered_words))
df_test = pd.DataFrame(testmeta1)
# df_test = df_test.ix[:, 0:6]
# df_test.columns = ['sample0', 'sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6']
# df_test = df_test.fillna("")
df_test

In [ ]:
arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample1'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor1'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample2'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor2'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample3'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor3'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample4'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor4'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample5'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor5'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample6'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor6'] = logreg_predictions

df_test['sum_metaphor'] = df_test['metaphor1'] + df_test['metaphor2'] + df_test['metaphor3'] +df_test['metaphor4'] +df_test['metaphor5'] 

if sum(df_test['sum_metaphor']) >= 1:
    df_test['sum_metaphor'] = 1
sum(df_test['sum_metaphor'])
#len(df_test['sum_metaphor'])
df_test

In [ ]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = ' '.join([word.lower() for word in word_list if word not in stopwords.words('english')])
        testmeta1.append(filtered_words)
df_test = pd.DataFrame(testmeta1)
df_test
# df_test = df_test.ix[:, 0:6]
df_test.columns = ['sample0']
df_test = df_test.fillna("")
# df_test

In [ ]:
arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

df_test

In [ ]:
df_test.ix[:, 0:5].head()

In [ ]:
len(df_test.columns)
#df_test[0]

In [ ]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = ' '.join([word.lower() for word in word_list if word not in stopwords.words('english')])
        testmeta1.append(filtered_words)
df_test = pd.DataFrame(testmeta1)
df_test
# df_test = df_test.ix[:, 0:6]
df_test.columns = ['sample0']
df_test = df_test.fillna("")
# df_test

In [ ]:
import nltk, re
from nltk.corpus import brown
from nltk import word_tokenize

In [ ]:
def create_data_sets(sentences):
    size = int(len(sentences) * 0.9)
    train_sents = sentences[:size]
    test_sents = sentences[size:]
    return train_sents, test_sents

def build_backoff_tagger (train_sents):
    t0 = nltk.DefaultTagger('NN')
    t1 = nltk.UnigramTagger(train_sents, backoff=t0)
    t2 = nltk.BigramTagger(train_sents, backoff=t1)
    return t2


def train_tagger(already_tagged_sents):
    train_sents, test_sents = create_data_sets(already_tagged_sents)
    ngram_tagger = build_backoff_tagger(train_sents)
    print ("%0.3f pos accuracy on test set" % ngram_tagger.evaluate(test_sents))
    return ngram_tagger


In [ ]:
def train_tagger_on_brown():
    brown_tagged_sents = brown.tagged_sents(categories=['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies',
    'humor', 'learned', 'lore', 'mystery', 'religion', 'reviews', 'romance','science_fiction'])
    return train_tagger(brown_tagged_sents)


In [ ]:
def tokenize_text(corpus):
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sents = sent_tokenizer.tokenize(corpus) # Split text into sentences    
    return [nltk.word_tokenize(word) for word in raw_sents]

def create_corpus(f):
    with open(f, 'r') as text_file:
        new_corpus = text_file.read()
    return new_corpus


In [ ]:
brown_tagger = train_tagger_on_brown()

In [ ]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    tokenizer = RegexpTokenizer(r'\w+')
    for line in testset:
        word_list = line.split()
#         filtered_words = ' '.join([word.lower() for word in word_list if word not in stopwords.words('english')])
        filtered_words = ' '.join([word.lower() for word in word_list])
        testmeta1.append(tokenizer.tokenize(filtered_words))
testmeta1[0]

def get_pos(sents, tagger):        
    return [tagger.tag(sent) for sent in sents]

pos = get_pos(testmeta1, brown_tagger)
pos

In [ ]:
def train_tagger_on_brown_augmented_with_additional_sents():

    additional_sents = [[('colorful', 'JJ'), ('quilt', 'NN')],
                        [('regions', 'NN'), ('represent', 'VB'), ('world', 'NN')],
                        [('public', 'NN'), ('explore', 'VB'), ('brain', 'NN')]]


    brown_tagged_sents = brown.tagged_sents(categories=['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies',
    'humor', 'learned', 'lore', 'mystery', 'religion', 'reviews', 'romance', 'science_fiction'])
    
    #append hand-tagged cooking sentences to the front of the training data
    all_tagged_sents = additional_sents + brown_tagged_sents
    return train_tagger(all_tagged_sents)

brown_and_additional_tagger = train_tagger_on_brown_augmented_with_cooking_sents()

In [ ]:
better_sentences = get_pos(testmeta1, brown_and_additional_tagger)

In [ ]:
with open("test_meta_sent.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    tokenizer = RegexpTokenizer(r'\w+')
    for line in testset:
        word_list = line.split()
#         filtered_words = ' '.join([word.lower() for word in word_list if word not in stopwords.words('english')])
        filtered_words = ' '.join([word.lower() for word in word_list])
        testmeta1.append(tokenizer.tokenize(filtered_words))
testmeta1[0]

def get_pos(sents, tagger):        
    return [tagger.tag(sent) for sent in sents]

pos = get_pos(testmeta1, brown_tagger)
pos

# Switch to Classification Features with NLTK Classification Code

In [ ]:
%matplotlib inline 
import nltk
from nltk.corpus import names
import random

In [ ]:
f = open('an_mets.txt', 'r')
x = f.readlines()
an_mets = [t.rstrip() for t in x[1::]]
f = open('an_nonmets.txt', 'r')
x = f.readlines()
an_nonmets = [t.rstrip() for t in x[1::]]
f = open('svo_mets.txt', 'r')
x = f.readlines()
svo_mets = [t.rstrip() for t in x[1::]]
f = open('svo_nonmets.txt', 'r')
x = f.readlines()
svo_nonmets = [t.rstrip() for t in x[1::]]

In [ ]:
# def double_letter(word):
#     letter_list = []
#     for letter in word:
#         if word.count(letter) > 1:
#             letter_list.append(True)
#         else:
#             letter_list.append(False)
#     if True in letter_list:
#         return True
#     else:
#         return False
    
def metaphor_features(word):
    features = {}
    word = word.lower()
    features['POS'] = word[0]
#     features['last'] = word[-1]
#     features['last 2'] = word[-2]
#     features['first 3'] = word[:3]
#     features['first'] = word[:1]
#     features['length'] = len(word)
#     features['starts with K'] = word.startswith('k')
#     features['ends with i'] = word.endswith('i')
#     features['ends with a'] = word.endswith('a')
#     features['double letter'] = double_letter(word)
    return features
print(str(metaphor_features('Michelle')))  

In [ ]:
def create_an_data():
    an_mets_tag = [(re.sub(r'[^\w\s]','',line.lower()), 'metaphor') for line in an_mets]
    an_nonmets_tag = [(re.sub(r'[^\w\s]','',line.lower()), 'NOT metaphor') for line in an_nonmets]
    svo_mets_tag = [(re.sub(r'[^\w\s]','',line.lower()), 'metaphor') for line in svo_mets]
    svo_nonmets_tag = [(re.sub(r'[^\w\s]','',line.lower()), 'NOT metaphor') for line in svo_nonmets]
    all_an = an_mets_tag + an_nonmets_tag + svo_mets_tag + svo_nonmets_tag
    
    # Randomize the order of male and female names, and de-alphabatize
    random.shuffle(all_an)
    return all_an

all_an_data = create_an_data()
# print(all_an_data)

In [ ]:
# This function allows experimentation with different feature definitions
# items is a list of (key, value) pairs from which features are extracted and training sets are made
# Feature sets returned are dictionaries of features

# This function also optionally returns the names of the training, development, 
# and test data for the purposes of error checking

def create_training_sets (feature_function, items, return_items=False):
    # Create the features sets.  Call the function that was passed in.
    # For names data, key is the name, and value is the gender
    featuresets = [(feature_function(key), value) for (key, value) in items]
    
    # Divided training and testing in thirds.  Could divide in other proportions instead.
    third = int(float(len(featuresets)) / 3.0)
    
    train_set, dev_set, test_set = featuresets[0:third], featuresets[third:third*2], featuresets[third*2:]
    train_items, dev_items, test_items = items[0:third], items[third:third*2], items[third*2:]
    if return_items == True:
        return train_set, dev_set, test_set, train_items, dev_items, test_items
    else:
        return train_set, dev_set, test_set

In [ ]:
# note that this passes in a function name as an argument (gender_features)

train_set, dev_set, test_set = create_training_sets(metaphor_features, all_an_data)
cl = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
print ("'white blanket': " + cl.classify(metaphor_features('white blanket')))
print ("'snow white': " + cl.classify(metaphor_features('snow white')))
print ("'white as snow': " + cl.classify(metaphor_features('white as snow')))
print ("'happy customer': " + cl.classify(metaphor_features('happy customer')))
print ("'boiling rage': " + cl.classify(metaphor_features('boiling rage')))

In [ ]:
testsample = 'one specific aspect'
print (testsample + ": " + cl.classify(metaphor_features(testsample)))

In [ ]:
print ("%.3f" % nltk.classify.accuracy(cl, dev_set))
cl.show_most_informative_features(15)

In [ ]:

with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [re.sub(r'[^\w\s]','',word.lower()) for word in word_list if word.lower() not in stopwords.words('english')]
        testmeta.append([' '.join(x) for x in trigrams(filtered_words)])
print(testmeta)

In [ ]:
with open("test_meta_sent.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [re.sub(r'[^\w\s]','',word.lower()) for word in word_list if word.lower() not in stopwords.words('english')]
        testmeta.append([' '.join(x) for x in trigrams(filtered_words)])
print(testmeta)

In [ ]:
metaphor_count = 0
nonmetaphor_count = 0
for x in testmeta:
    for y in x:
        testsample = y
        print (testsample + ": " + cl.classify(metaphor_features(testsample)))
        if cl.classify(metaphor_features(testsample)) == 'metaphor':
            metaphor_count += 1
        elif cl.classify(metaphor_features(testsample)) == 'NOT metaphor':
            nonmetaphor_count += 1
            
print('metaphor_count:' + str(metaphor_count))
print('nonmetaphor_count:' + str(nonmetaphor_count))

In [ ]:
def read_article_for_metaphor(article):
    with open("sciencearticle.txt", "r") as testset:
        filtered_words = []
        testmeta = []
        word_list = ''
        para_index = 0
    #     text = ''.join(testset.readlines())
    #     sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)
    #     sentences = [t.rstrip() for t in sentences]
    #     print(sentences)
        for line in testset:
            para_index += 1
            word_list = line.split()
            filtered_words = [re.sub(r'[^\w\s]','',word.lower()) for word in word_list if word not in stopwords.words('english')]
            testmeta.append([' '.join(x) for x in trigrams(filtered_words)])
            print(para_index)
    # print(testmeta)

    metaphor_count = 0
    nonmetaphor_count = 0
    metaphor_list = []
    for x in testmeta:
        for y in x:
            testsample = y
            print (testsample + ": " + cl.classify(metaphor_features(testsample)))
            if cl.classify(metaphor_features(testsample)) == 'metaphor':
                metaphor_count += 1
                metaphor_list.append(testsample)
            elif cl.classify(metaphor_features(testsample)) == 'NOT metaphor':
                nonmetaphor_count += 1

    print('metaphor_count:' + str(metaphor_count))
    print('nonmetaphor_count:' + str(nonmetaphor_count))
    print(metaphor_list)

In [ ]:
with open("sciencearticle.txt", "r") as testset:
    word_list = ''
    highlight_bucket = []
    for line in testset:
        print(line)
        for trigram in metaphor_list:
            trigram_list = trigram.split()
            index = line.find(trigram_list[0])
            index_len = len(trigram)
            highlight_bucket.append(line[index:index+index_len])
print(highlight_bucket)

In [ ]:
sample = ['the', 'methods', 'i', 'am', 'michelle']
# word_list
word_str = ' '.join(word_list)
# print(word_str)
for word in sample:
    print(word_str.find(word))

In [ ]:
def create_regex_strings():
    stopwords_regex = "("
    # print(stopwords.words('english'))
    for word in stopwords.words('english'):
        stopwords_regex = stopwords_regex + word + "|"
    stopwords_regex = stopwords_regex[:-1] + "|an)"
#     print(stopwords_regex)
    punctuation_regex = r"[“”!\"#$%&'\(\)*+,-./:;<=>?@^_`{}~\s]*"
    return stopwords_regex, punctuation_regex

# 


In [ ]:
# with open("sciencearticle.txt", "r") as testset:
def Rule1_metaphor(article):
    article = article.lower()
    word_list = ''
    highlight_bucket = []
    tag_list = []
    stopwords_regex, punctuation_regex = create_regex_strings()
    
    for trigram in metaphor_list:
#         print(trigram)
        trigram_list = trigram.split(" ")
        regex_string = "(" + trigram_list[0] + punctuation_regex + "(" + stopwords_regex + punctuation_regex + ")*" + trigram_list[1] + punctuation_regex + "(" + stopwords_regex + punctuation_regex + ")*" + trigram_list[2] + ")" 
#         print(regex_string)
#         print(article)
        match = re.search(regex_string, article)
        if match is not None:
            complete_trigram = match.group(0)
#             print(trigram, complete_trigram)
            index = article.find(complete_trigram)
#             print(index, complete_trigram)
            index_len = len(trigram)
            highlight_bucket.append(article[index:index+index_len])
            tag_list.append((index, index_len))
#             article = article[index+index_len:]
        else:
             print("Not found: " + trigram)
    
#     for line in testset:
# #         print(line)
#         for trigram in metaphor_list:
#             trigram_list = trigram.split()
#             index = line.find(trigram_list[0])
#             index_len = len(trigram)
#             highlight_bucket.append(line[index:index+index_len])
#             line = line[index+index_len:]
#         print(highlight_bucket)
#         len(highlight_bucket)
        
    return tag_list
with open("sciencearticle.txt", "r") as testset:   
#     print(metaphor_list)
    print(Rule1_metaphor(testset.read()))

In [ ]:
with open("sciencearticle.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    word_list = ''
    para_index = 0
#     text = ''.join(testset.readlines())
#     sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)
#     sentences = [t.rstrip() for t in sentences]
#     print(sentences)
    for line in testset:
        para_index += 1
        word_list = line.split()
        filtered_words = [re.sub(r'[^\w\s]','',word.lower()) for word in word_list if word not in stopwords.words('english')]
        testmeta.append([' '.join(x) for x in trigrams(filtered_words)])
        print(para_index)
# print(testmeta)

metaphor_count = 0
nonmetaphor_count = 0
metaphor_list = []
for x in testmeta:
    for y in x:
        testsample = y
        print (testsample + ": " + cl.classify(metaphor_features(testsample)))
        if cl.classify(metaphor_features(testsample)) == 'metaphor':
            metaphor_count += 1
            metaphor_list.append(testsample)
        elif cl.classify(metaphor_features(testsample)) == 'NOT metaphor':
            nonmetaphor_count += 1
            
print('metaphor_count:' + str(metaphor_count))
print('nonmetaphor_count:' + str(nonmetaphor_count))
print(metaphor_list)